In [48]:
import ollama
def call_ollama(prompt):
    response = ollama.chat(model='qwen2.5:latest', messages=[{"role": "user", "content": prompt}])
    return response['message']['content']


In [101]:
"delighted".lower()

'delighted'

In [102]:
def check_rule(statement: str) -> str:
    return f"[Rule Check] '{statement}' fits mythological logic."

rule_tool = {
    "name": "rule_checker",
    "func": check_rule,
    "description": "Check whether a statement conforms to Shan Hai Jing mythological laws."
}


class NPCAgent:
    def __init__(self, npc_name: str):
        with open("app/data/characters.json", "r", encoding="utf-8") as f:
            characters = json.load(f)
        self.data = characters[npc_name]

        # role_playing
        self.name = self.data["id"]
        self.persona = self.data["system_prompt"]
        self.appearance = self.data['appearance']
        self.ability = self.data["abilities"]
        # self.purpose = self.data["purpose"]
        self.emotion = self.data["emotion_state"]
        self.agent = self

        # thinking style (perception+reflection->reasoning+planning)
        self.prompt_template = (
            "You are {self.name}.\nYou are currently participating in a mythological court trial "

            "### Scene Context ###\n"
            "{context}\n\n"

            "### Speaking Rules ###\n"
            "1. Speak **briefly and to the point**. Most of your replies should be within 1–3 sentences.\n"
            "2. Only speak at length (up to 4–5 sentences) if your personal interests, survival, or honor are directly challenged.\n"
            "3. If the Judge asks for a fact, answer **clearly and concisely**, without emotional elaboration.\n"
            "4. If the Judge accuses you or another NPC, you may defend or refute with emotional tone, "
            "5. Never repeat previous information unless it changes the context or contradicts new evidence.\n"
            "6. Avoid giving monologues; this is a court with turn-based dialogue, so keep your reply short.\n\n"

            "### Output Requirements ###\n"
            "- Keep your reply under 80 words (around 3–5 short sentences max).\n"
            "- Stay consistent with your memory and tone.\n"
            "- Mention details from your own memories if they help clarify the case.\n\n"
            "Now, speak as {persona}, responding to the Judge or the court.\n"
        )


    def analyze_emotion(self, dialogue):

        prompt = f"""
            You are a court observer. Please speak according to the content and tone of the following roles
            To determine the current psychological state of this character, you can only choose one of the following four options:
            [ordinary, guilty, angry, delighted]

            my personality: {self.persona},
            conversation: {dialogue}

            Only output the emotion words themselves and do not explain.
            Output example: Ordinary
        """
        try:
            result = call_ollama(prompt)
        except Exception as e:
            result = f"(error: {e})"

        result = result.strip().replace(":", "").replace(":", "").split()[0]
        if result.lower() not in ["ordinary", "guilty", "angry", "delighted"]:
            result = "ordinary"
        return result.lower()

    def invoke(self, inputs: dict):
        context = inputs.get("context", "")
        user_input = inputs.get("input", "")

        prompt = self.prompt_template.format(
            context=context,
            input=user_input
        )

        try:
            output = call_ollama(prompt).strip()
        except Exception as e:
            output = f"(model error: {e})"

        self.chat_history.append({"input": user_input, "output": output})
        return {"output": output}


    # action
    def speak(self, event: dict) -> str:
        context = (
            f"Event name: {event.get('name')}\n"
            f"Description: {event.get('description')}\n"
            f"Recent logs: {event.get('game_logs', [])}\n"
            f"Involved NPCs: {event.get('npcs', [])}"
        )

        last_input = ""
        if event.get("game_logs"):
            for entry in reversed(event["game_logs"]):
                for k, v in entry.items():
                    if "Judge" in k:
                        last_input = v
                        break
                if last_input:
                    break

        try:
            result = self.invoke({
                "persona": self.persona,
                "context": context,
                "input": last_input
            })
            response = result.get("output", "").strip()
        except Exception as e:
            response = f"(system error: {e})"

        self.emotion = self.analyze_emotion(response)
        print(f"[Emotion: {self.emotion}]")

        return response

# npc = NPCAgent("kui")
# response = npc.speak(event)
# print(response)

In [103]:
# need choose npc before game
class gameAgent:
    def __init__(self, npc_list, event):

        # event
        self.event = event
        self.public_clue = "\n".join(f"{idx}. {c['clue']}" for idx, c in enumerate(event["p_clues"], start=1))
        self.public_dialogue = event['game_logs']

        # characters
        with open("app/data/characters.json", "r", encoding="utf-8") as f:
            characters = json.load(f)
        if npc_list == None:
            self.npc_list = event['npcs']
        else:
            self.npc_list = npc_list
        self.visual = {char_id: char_data["appearance"] for char_id, char_data in characters.items()}
        self.ability = {char_id: char_data["abilities"] for char_id, char_data in characters.items()}

        # game_status
        self.trail = True

    def process(self):
        print(f"Chapter 1 - Event trail start: {self.event['name']}")
        print(f"Clue Board:\n{self.public_clue}")

    # globally decide who speak next
    def pick_next_speaker(self):
        context = f"""
            The Event name: {self.event['name']}.
            The Case Description: {self.event['description']}.
            Previous Game Conversation: {self.public_dialogue}
        """
        result = call_ollama(f"""
            You are a game organizer. 
            Based on the previous game conversation content, select the next speaker from the {self.npc_list}. 
            You should prioritize Judge's requirements.
            If the judge does not explicitly point it out, analyze through the content of the conversation who is more suitable to speak at this time. 
            It might have been mentioned by another character, or the content of the conversation implies that it is related to the character.
            If there is no clear direction, please analyze the dialogue content and select the speaker who can best provoke the plot twist.
            One character cannot continuously speak more than 2 rounds.
            Dialogue content:{context}, npc_list:{self.npc_list}, please only output the name of your choice, for example: 'Next speaker: Baize'.
            Next speaker: 
        """)
        last_index = -1
        last_npc = self.npc_list[0]
        for npc in self.npc_list:
            idx = str(result).lower().rfind(npc)
            if idx > last_index:
                last_index = idx
                last_npc = npc
        return last_npc

    # user action
    def user_act(self, choice):
        if choice == 1: # pass to agent
            cur_npc = self.pick_next_speaker()
        elif choice == 2: # choose char to speak
            lines = " ".join(f"{idx}. {c}" for idx, c in enumerate(self.npc_list, start=1))
            print(f"please choose the speaker:\n{lines}")
            idx = input("input the speaker index number (1 or 2 or 3...): ").strip()
            cur_npc = self.npc_list[int(idx) - 1]
        elif choice == 3: # choose to speak
            user_dia = input("please say something: ").strip()
            self.public_dialogue.append({"Judge(player)":f"{user_dia}"})
            cur_npc = self.pick_next_speaker()
        else:
            print("User action wrong - user_act")
        return cur_npc

    # agent action
    def char_act(selfcur_npc):
        char_dia = speak(cur_npc)
        self.public_dialogue.append({f"{cur_npc}":f"{char_dia}"})
        return event

    def speak(cur_npc):
        npc = NPCAgent(cur_npc)
        char_dia = npc.speak(event)
        print(char_dia)
        return char_dia

# import random
# while trail_start:
#     choice = random.choice([1,2,3])
#     event, cur_npc = user_act(event, choice)
#     event = char_act(cur_npc)

In [104]:
def setup_trial(event):
    # initialization
    game = gameAgent(event["npcs"], event)
    game.process()

    # create character AI
    npc_pool = {name: NPCAgent(name) for name in game.npc_list}
    print(f"All NPCs initialized: {list(npc_pool.keys())}")

    return game, npc_pool


# main loop
def run_trial(event):
    game, npc_pool = setup_trial(event)
    round_count = 0

    while game.trail:
        print(f"\n---- Trial Round {round_count + 1} ----")

        # user action: 1,2,3
        choice = random.choice([1, 3])  # simulate 这个要改
        cur_npc = game.user_act(choice)

        # NPC speak
        if cur_npc not in npc_pool:
            print(f"[System] {cur_npc} not found in npc_pool.")
            continue

        npc = npc_pool[cur_npc]
        npc_dialogue = npc.speak(game.event)

        # publish to global
        game.public_dialogue.append({cur_npc: npc_dialogue})
        game.event["game_logs"].append({cur_npc: npc_dialogue})
        print(f"\n[{cur_npc.lower()}]: {npc_dialogue}\n")

        # out loop
        round_count += 1
        if round_count > 5:
            result = call_ollama(f"""
                You are the trial narrator. 
                The following is the dialogue so far:
                {game.public_dialogue}
                Determine if the killer has been revealed or the case should end.
                Respond with 'continue' or 'end'.
            """)
            if "end" in result.lower():
                game.trail = False
                print(">>> The trial has ended (LLM decided).")
                break


    return game.public_dialogue


In [105]:
with open("app/data/events.json", "r", encoding="utf-8") as f:
    event = json.load(f)[0]    
with open("app/data/characters.json", "r", encoding="utf-8") as f:
    characters = json.load(f)
npc_list = event['npcs']

dialogue_log = run_trial(event)
for line in dialogue_log:
    print(line)

Chapter 1 - Event trail start: Eclipse Festival and Nine-Headed Killing: The nine-headed snake does not speak; all who speak are snakes.
Clue Board:
1. Blue phosphorescence remains on the altarpiece.
2. There were broken feathers and claw marks on the ground, but their directions were disordered.
3. The only witness claimed to have seen "multiple shadows" sliding.
All NPCs initialized: ['jiuweihu', 'yingzhao', 'kui', 'bifang', 'xingxing', 'qiongqi', 'qingniao']

---- Trial Round 1 ----
[Emotion: ordinary]

[qingniao]: (system error: 'self')


---- Trial Round 2 ----


KeyboardInterrupt: 